In [1]:
!pip install -U Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U git+https://github.com/scikit-multiflow/scikit-multiflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/scikit-multiflow/scikit-multiflow to /tmp/pip-req-build-59j5e4k0
  Running command git clone -q https://github.com/scikit-multiflow/scikit-multiflow /tmp/pip-req-build-59j5e4k0


In [3]:
%matplotlib notebook

In [30]:
import numpy.core.multiarray
import numpy as np
from skmultiflow.data import FileStream
from skmultiflow.transform import WindowedMinmaxScaler
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.lazy import SAMKNNClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.meta import LeveragingBaggingClassifier
from skmultiflow.lazy import KNNClassifier
from skmultiflow.drift_detection.adwin import ADWIN
from skmultiflow.evaluation import EvaluatePrequential

In [64]:
url_abrupt = 'https://raw.githubusercontent.com/durga256/U_ML_OnlineLearning/main/INSECTS-abrupt_balanced_norm.csv'
url_gradual = 'https://raw.githubusercontent.com/durga256/U_ML_OnlineLearning/main/INSECTS-gradual_balanced_norm.csv'
url_incremental = 'https://raw.githubusercontent.com/durga256/U_ML_OnlineLearning/main/INSECTS-incremental_balanced_norm.csv'
stream_abrupt = FileStream(url_abrupt,cat_features=[-1])
stream_gradual = FileStream(url_gradual,cat_features=[-1])
stream_incremental = FileStream(url_incremental,cat_features=[-1])

In [56]:
#nochange classifier
stream_abrupt.restart()
features, target = stream_abrupt.next_sample()
current_state = target[0]
total_count = 1
total_right = 0
accuracy_list = []
while stream_abrupt.has_more_samples():
  features, target = stream_abrupt.next_sample()
  if current_state == target[0]:
    total_right += 1
  current_state = target[0]
  total_count += 1
  if total_count == 1000:
    accuracy_list.append(total_right/total_count)
    total_count = 0
    total_right = 0

np.savetxt('output_abrupt_NC.csv', (accuracy_list), delimiter=',')
sum(accuracy_list)/len(accuracy_list)

0.2784615384615384

In [57]:
#nochange classifier
stream_gradual.restart()
features, target = stream_gradual.next_sample()
current_state = target[0]
total_count = 1
total_right = 0
accuracy_list = []
while stream_gradual.has_more_samples():
  features, target = stream_gradual.next_sample()
  if current_state == target[0]:
    total_right += 1
  current_state = target[0]
  total_count += 1
  if total_count == 1000:
    accuracy_list.append(total_right/total_count)
    total_count = 0
    total_right = 0

np.savetxt('output_gradual_NC.csv', (accuracy_list), delimiter=',')
sum(accuracy_list)/len(accuracy_list)

0.3619583333333334

In [58]:
#nochange classifier
stream_incremental.restart()
features, target = stream_incremental.next_sample()
current_state = target[0]
total_count = 1
total_right = 0
accuracy_list = []
while stream_incremental.has_more_samples():
  features, target = stream_incremental.next_sample()
  if current_state == target[0]:
    total_right += 1
  current_state = target[0]
  total_count += 1
  if total_count == 1000:
    accuracy_list.append(total_right/total_count)
    total_count = 0
    total_right = 0

np.savetxt('output_incremental_NC.csv', (accuracy_list), delimiter=',')
sum(accuracy_list)/len(accuracy_list)

0.1608070175438596

In [60]:
#majority class classifier
stream_abrupt.restart()
features, target = stream_abrupt.next_sample()
max_list = [0,0,0,0,0,0] #for 6 target classes
accuracy_list = []
while stream_abrupt.has_more_samples():
  features, target = stream_abrupt.next_sample()
  if max_list.index(max(max_list)) == target[0]:
    total_right += 1
  max_list[target[0]] += 1
  total_count += 1
  if total_count%1000 == 0:
    accuracy_list.append(total_right/total_count)
    max_list=[0,0,0,0,0,0]
    total_count = 0
    total_right = 0

np.savetxt('output_abrupt_MC.csv', (accuracy_list), delimiter=',')
sum(accuracy_list)/len(accuracy_list)

0.3139811320754717

In [61]:
#majority class classifier
stream_gradual.restart()
features, target = stream_gradual.next_sample()
max_list = [0,0,0,0,0,0] #for 6 target classes
accuracy_list = []
while stream_gradual.has_more_samples():
  features, target = stream_gradual.next_sample()
  if max_list.index(max(max_list)) == target[0]:
    total_right += 1
  max_list[target[0]] += 1
  total_count += 1
  if total_count%1000 == 0:
    accuracy_list.append(total_right/total_count)
    max_list=[0,0,0,0,0,0]
    total_count = 0
    total_right = 0

np.savetxt('output_gradual_MC.csv', (accuracy_list), delimiter=',')
sum(accuracy_list)/len(accuracy_list)

0.37266666666666665

In [62]:
#majority class classifier
stream_incremental.restart()
features, target = stream_incremental.next_sample()
max_list = [0,0,0,0,0,0] #for 6 target classes
accuracy_list = []
while stream_incremental.has_more_samples():
  features, target = stream_incremental.next_sample()
  if max_list.index(max(max_list)) == target[0]:
    total_right += 1
  max_list[target[0]] += 1
  total_count += 1
  if total_count%1000 == 0:
    accuracy_list.append(total_right/total_count)
    max_list=[0,0,0,0,0,0]
    total_count = 0
    total_right = 0

np.savetxt('output_incremental_MC.csv', (accuracy_list), delimiter=',')
sum(accuracy_list)/len(accuracy_list)

0.13792982456140349

In [6]:
classifier = [HoeffdingTreeClassifier(), SAMKNNClassifier(max_window_size=1000), HoeffdingAdaptiveTreeClassifier()]

In [67]:
eval_abrupt = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, metrics=['accuracy'], output_file="output_abrupt_HT_SAMKNN_HAT.csv")
eval_gradual = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, metrics=['accuracy'], output_file="output_gradual_HT_SAMKNN_HAT.csv")
eval_incremental = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, metrics=['accuracy'], output_file="output_incremental_HT_SAMKNN_HAT.csv")

In [68]:
eval_abrupt.evaluate(stream=stream_abrupt, model=classifier, model_names=['HT', 'SAMKNN', 'HAT'])

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [98%] [257.40s]
Processed samples: 53000
Mean performance:
HT - Accuracy     : 0.5140
SAMKNN - Accuracy     : 0.6464
HAT - Accuracy     : 0.5775


[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nba', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False, nominal_attributes=None,
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 SAMKNNClassifier(ltm_size=0.4, max_window_size=None, min_stm_size=50,
                  n_neighbors=5, stm_size_option='maxACCApprox', use_ltm=True,
                  weighting='distance'),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_prepru

In [69]:
eval_gradual.evaluate(stream=stream_gradual, model=classifier, model_names=['HT', 'SAMKNN', 'HAT'])

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [99%] [134.03s]
Processed samples: 25000
Mean performance:
HT - Accuracy     : 0.5080
SAMKNN - Accuracy     : 0.6702
HAT - Accuracy     : 0.6036


[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nba', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False, nominal_attributes=None,
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 SAMKNNClassifier(ltm_size=0.4, max_window_size=None, min_stm_size=50,
                  n_neighbors=5, stm_size_option='maxACCApprox', use_ltm=True,
                  weighting='distance'),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_prepru

In [70]:
eval_incremental.evaluate(stream=stream_incremental, model=classifier, model_names=['HT', 'SAMKNN', 'HAT'])

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [100%] [304.52s]
Processed samples: 58000
Mean performance:
HT - Accuracy     : 0.5315
SAMKNN - Accuracy     : 0.5659
HAT - Accuracy     : 0.5420


[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nba', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False, nominal_attributes=None,
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 SAMKNNClassifier(ltm_size=0.4, max_window_size=None, min_stm_size=50,
                  n_neighbors=5, stm_size_option='maxACCApprox', use_ltm=True,
                  weighting='distance'),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_prepru

In [71]:
classifier_arf = AdaptiveRandomForestClassifier()

In [73]:
evaluator_abrupt = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, output_file="output_abrupt_arf.csv",
                                show_plot=True, metrics=['accuracy'])

evaluator_abrupt.evaluate(stream=stream_abrupt, model=classifier_arf)

<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...

Time limit reached (1000.00s). Evaluation stopped.
Processed samples: 41000
Mean performance:
M0 - Accuracy     : 0.6687


[AdaptiveRandomForestClassifier(binary_split=False, disable_weighted_vote=False,
                                drift_detection_method=ADWIN(delta=0.001),
                                grace_period=50, lambda_value=6,
                                leaf_prediction='nba', max_byte_size=33554432,
                                max_features=6, memory_estimate_period=2000000,
                                n_estimators=10, nb_threshold=0,
                                no_preprune=False, nominal_attributes=None,
                                performance_metric='acc', random_state=None,
                                remove_poor_atts=False, split_confidence=0.01,
                                split_criterion='info_gain',
                                stop_mem_management=False, tie_threshold=0.05,
                                warning_detection_method=ADWIN(delta=0.01))]

In [77]:
evaluator_gradual = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, output_file="output_gradual_arf.csv",
                                show_plot=True, metrics=['accuracy'])

evaluator_gradual.evaluate(stream=stream_gradual, model=classifier_arf)

<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [99%] [576.03s]
Processed samples: 25000
Mean performance:
M0 - Accuracy     : 0.7200


[AdaptiveRandomForestClassifier(binary_split=False, disable_weighted_vote=False,
                                drift_detection_method=ADWIN(delta=0.001),
                                grace_period=50, lambda_value=6,
                                leaf_prediction='nba', max_byte_size=33554432,
                                max_features=6, memory_estimate_period=2000000,
                                n_estimators=10, nb_threshold=0,
                                no_preprune=False, nominal_attributes=None,
                                performance_metric='acc', random_state=None,
                                remove_poor_atts=False, split_confidence=0.01,
                                split_criterion='info_gain',
                                stop_mem_management=False, tie_threshold=0.05,
                                warning_detection_method=ADWIN(delta=0.01))]

In [78]:
evaluator_incremental = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, output_file="output_incremental_arf.csv",
                                show_plot=True, metrics=['accuracy'])

evaluator_incremental.evaluate(stream=stream_incremental, model=classifier_arf)

<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...

Time limit reached (1000.00s). Evaluation stopped.
Processed samples: 35000
Mean performance:
M0 - Accuracy     : 0.6108


[AdaptiveRandomForestClassifier(binary_split=False, disable_weighted_vote=False,
                                drift_detection_method=ADWIN(delta=0.001),
                                grace_period=50, lambda_value=6,
                                leaf_prediction='nba', max_byte_size=33554432,
                                max_features=6, memory_estimate_period=2000000,
                                n_estimators=10, nb_threshold=0,
                                no_preprune=False, nominal_attributes=None,
                                performance_metric='acc', random_state=None,
                                remove_poor_atts=False, split_confidence=0.01,
                                split_criterion='info_gain',
                                stop_mem_management=False, tie_threshold=0.05,
                                warning_detection_method=ADWIN(delta=0.01))]

In [74]:
classifier_LB = LeveragingBaggingClassifier(base_estimator=KNNClassifier(max_window_size=1000))

In [76]:
evaluator_abrupt_lb = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, output_file="output_abrupt_LB.csv",
                                show_plot=True, metrics=['accuracy'])

evaluator_abrupt_lb.evaluate(stream=stream_abrupt, model=classifier_LB)

<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...

Time limit reached (1000.00s). Evaluation stopped.
Processed samples: 23000
Mean performance:
M0 - Accuracy     : 0.5781


[LeveragingBaggingClassifier(base_estimator=KNNClassifier(leaf_size=30,
                                                          max_window_size=1000,
                                                          metric='euclidean',
                                                          n_neighbors=5),
                             delta=0.002, enable_code_matrix=False,
                             leverage_algorithm='leveraging_bag',
                             n_estimators=10, random_state=None, w=6)]

In [79]:
evaluator_gradual_lb = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, output_file="output_gradual_LB.csv",
                                show_plot=True, metrics=['accuracy'])

evaluator_gradual_lb.evaluate(stream=stream_gradual, model=classifier_LB)

<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...

Time limit reached (1000.00s). Evaluation stopped.
Processed samples: 24000
Mean performance:
M0 - Accuracy     : 0.6307


[LeveragingBaggingClassifier(base_estimator=KNNClassifier(leaf_size=30,
                                                          max_window_size=1000,
                                                          metric='euclidean',
                                                          n_neighbors=5),
                             delta=0.002, enable_code_matrix=False,
                             leverage_algorithm='leveraging_bag',
                             n_estimators=10, random_state=None, w=6)]

In [80]:
evaluator_incremental_lb = EvaluatePrequential(pretrain_size=1000, batch_size=1000,max_time=1000, output_file="output_incremental_LB.csv",
                                show_plot=True, metrics=['accuracy'])

evaluator_incremental_lb.evaluate(stream=stream_incremental, model=classifier_LB)

<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...

Time limit reached (1000.00s). Evaluation stopped.
Processed samples: 24000
Mean performance:
M0 - Accuracy     : 0.4655


[LeveragingBaggingClassifier(base_estimator=KNNClassifier(leaf_size=30,
                                                          max_window_size=1000,
                                                          metric='euclidean',
                                                          n_neighbors=5),
                             delta=0.002, enable_code_matrix=False,
                             leverage_algorithm='leveraging_bag',
                             n_estimators=10, random_state=None, w=6)]

In [86]:
drift_detector = ADWIN()
list_change_abrupt = []
stream_abrupt.restart()
count = 1
while stream_abrupt.has_more_samples():
  features, target = stream_abrupt.next_sample()
  #for i, val in enumerate(features[0]):
  drift_detector.add_element(target[0])
  if drift_detector.detected_change():
    list_change_abrupt.append(count)
    drift_detector.reset()
  count += 1

list_change_abrupt

[12960, 14400, 18464, 19680, 28576, 35744, 38272, 38720, 40256, 46976, 52608]

In [84]:
drift_detector = ADWIN()
list_change_gradual = []
stream_gradual.restart()
count = 1
while stream_gradual.has_more_samples():
  features, target = stream_gradual.next_sample()
  drift_detector.add_element(target[0])
  if drift_detector.detected_change():
    list_change_gradual.append(count)
    drift_detector.reset()
  count += 1

list_change_gradual

[3328, 10656, 11872, 13824, 13920, 13984, 23040, 23328]

In [103]:
drift_detector = ADWIN()
list_change_incremental = []
stream_incremental.restart()
count = 1
while stream_incremental.has_more_samples():
  features, target = stream_incremental.next_sample()
  #for i, val in enumerate(features[0]):
  drift_detector.add_element(target[0])
  if drift_detector.detected_change():
    list_change_incremental.append(count)
    drift_detector.reset()
  count += 1

list_change_incremental

[]

In [100]:
np.savetxt('Drift_Detection_abrupt.csv', (list_change_abrupt), delimiter=',')

In [101]:
np.savetxt('Drift_Detection_gradual.csv', (list_change_gradual), delimiter=',')

In [102]:
np.savetxt('Drift_Detection_incremental.csv', (list_change_incremental), delimiter=',')